In [1]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

from datetime import date
data_atual = date.today()

In [2]:
formato_numerico = "{0:10.2f}"
ativo = list(())
AtivoSymbolInfo = list(())
matriz = list(())
global matriz

In [3]:
def imprimirInfoAtivo(t, y, n):
    print(t, y, formato_numerico.format(n))

In [4]:
# ****************************************************
def RolarOpcoes(ativo):
    n = len(ativo)
    AtivoSymbolInfo.clear()
    rows,cols = (n,10)
    matriz = [[0 for i in range(cols)] for j in range(rows)]
    
    for i in range(n):
        AtivoSymbolInfo.append(mt5.symbol_info(ativo[i]))
        xx = AtivoSymbolInfo[i]
        matriz[i][0] = xx.name
        matriz[i][1] = xx.option_strike
        matriz[i][4] = xx.last
 
        # somente para linha 1, que é da opção vendida, que preciso rolar
        # pega o ASK... mas se não tiver ninguém oferecendo, pega o LAST
        if i == 1:
            matriz[1][3] = xx.ask
            if matriz[1][3] == 0:
                fff = input('Entre o valor do Ask ' + xx.name + ': ')
                matriz[i][3] = float(fff)
            gasto = matriz[1][3] * -1
#            if matriz[1][3] == 0:       gasto = matriz[1][4]
 
        # para as linhas 2,3,... ou seja, para as opções dos meses seguintes que poderei vender
        if i>1:
            matriz[i][2] = xx.bid
            if matriz[i][2] == 0:
                fff = input('Entre o valor do Bid ' + xx.name + ': ')
                matriz[i][2] = float(fff)
            matriz[i][5] = matriz[i][1] - matriz[1][1] # aumento em R$ strike nova - strike vendida
            matriz[i][6] = round((matriz[i][1]/matriz[1][1] - 1) * 100, 2)  # % strike nova - strike vendido
            matriz[i][7] = matriz[i][5] + gasto + matriz[i][2] # dif.strike - gasto
            matriz[i][8] = round(matriz[i][7] / matriz[1][1] * 100, 2)
            matriz[i][9] = matriz[i][2] - matriz[1][3] 

    df = pd.DataFrame(matriz, columns=['Ativo','Strike','bid','ask','Last','▲ R$','▲ % Strike','Somatório','% sobre '+matriz[1][0], 'Cash'])
    print(df)
    nome_arq = 'rolar_opcoes_' + ativo[0] + '_' + ativo[1] + '_' + ativo[2]+ '.xlsx'
    df.to_excel(nome_arq, sheet_name=str(data_atual))
    

In [5]:
print(type(data_atual))

<class 'datetime.date'>


In [6]:
### https://www.alura.com.br/artigos/lidando-com-datas-e-horarios-no-python?gclid=CjwKCAjwieuGBhAsEiwA1Ly_nWDbtS13ZSnMrNsQGuuQCRKckt2Jbyv8byErnftNStX5-SAf5fxwSxoCpF8QAvD_BwE

In [7]:
data_em_texto = '{}/{}/{}'.format(data_atual.day, data_atual.month,
data_atual.year)
data_em_texto

'6/7/2021'

In [8]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike  bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.0  0.00  12.05  0.00        0.00       0.00   
1  CMIGG154   13.35  0.0  0.03   0.02  0.00        0.00       0.00   
2  CMIGH139   14.00  0.1  0.00   0.12  0.65        4.87       0.72   

   % sobre CMIGG154  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              5.39  0.07  


In [9]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH134'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  12.05  0.00        0.00       0.00   
1  CMIGG154   13.35  0.00  0.03   0.02  0.00        0.00       0.00   
2  CMIGH134   13.50  0.15  0.00   0.21  0.15        1.12       0.27   

   % sobre CMIGG154  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              2.02  0.12  


In [10]:
ativo = ['CMIG4', 'CMIGG154', 'CMIGH131', 'CMIGH134', 'CMIGH139'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CMIG4    0.00  0.00  0.00  12.05  0.00        0.00       0.00   
1  CMIGG154   13.35  0.00  0.03   0.02  0.00        0.00       0.00   
2  CMIGH131   13.25  0.20  0.00   0.18 -0.10       -0.75       0.07   
3  CMIGH134   13.50  0.15  0.00   0.21  0.15        1.12       0.27   
4  CMIGH139   14.00  0.10  0.00   0.12  0.65        4.87       0.72   

   % sobre CMIGG154  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              0.52  0.17  
3              2.02  0.12  
4              5.39  0.07  


In [11]:
ativo = ['ELET3', 'ELETG454', 'ELETH460', 'ELETH467', 'ELETH470', 'ELETH477', 'ELETH480'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     ELET3    0.00  0.00  0.00  42.70  0.00        0.00       0.00   
1  ELETG454   45.61  0.00  0.15   0.10  0.00        0.00       0.00   
2  ELETH460   46.00  1.13  0.00   1.15  0.39        0.86       1.37   
3  ELETH467   46.75  0.92  0.00   0.94  1.14        2.50       1.91   
4  ELETH470   47.00  0.86  0.00   0.90  1.39        3.05       2.10   
5  ELETH477   47.75  0.72  0.00   1.50  2.14        4.69       2.71   
6  ELETH480   48.00  0.75  0.00   0.77  2.39        5.24       2.99   

   % sobre ELETG454  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              3.00  0.98  
3              4.19  0.77  
4              4.60  0.71  
5              5.94  0.57  
6              6.56  0.60  


In [12]:
ativo = ['ITSA4', 'ITSAG118', 'ITSAH118'] 
RolarOpcoes(ativo)

      Ativo  Strike  bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     ITSA4    0.00  0.0  0.0  11.06  0.00        0.00       0.00   
1  ITSAG118   11.76  0.0  1.0   0.00  0.00        0.00       0.00   
2  ITSAH118   11.79  0.2  0.0   0.20  0.03        0.26      -0.77   

   % sobre ITSAG118  Cash  
0              0.00   0.0  
1              0.00   0.0  
2             -6.55  -0.8  


In [13]:
ativo = ['CSAN3', 'CSANG100', 'CSANH250'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     CSAN3    0.00  0.00  0.00  24.39  0.00        0.00        0.0   
1  CSANG100   24.74  0.00  0.44   0.31  0.00        0.00        0.0   
2  CSANH250   25.00  0.88  0.00   0.94  0.26        1.05        0.7   

   % sobre CSANG100  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              2.83  0.44  


In [14]:
ativo = ['BBSE3', 'BBSET220', 'BBSET230'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     BBSE3     0.0  0.00  0.00  22.73   0.0        0.00       0.00   
1  BBSET220    22.0  0.00  0.39   0.38   0.0        0.00       0.00   
2  BBSET230    23.0  0.77  0.00   0.79   1.0        4.55       1.38   

   % sobre BBSET220  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              6.27  0.38  


In [15]:
ativo = ['IRBR3', 'IRBRS600', 'IRBRT600'] 
RolarOpcoes(ativo)

      Ativo  Strike   bid   ask  Last  ▲ R$  ▲ % Strike  Somatório  \
0     IRBR3     0.0  0.00  0.00  5.66   0.0         0.0       0.00   
1  IRBRS600     6.0  0.00  0.38  0.38   0.0         0.0       0.00   
2  IRBRT600     6.0  0.42  0.00  0.45   0.0         0.0       0.04   

   % sobre IRBRS600  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              0.67  0.04  


In [38]:
RolarOpcoes(['VALE3', 'VALEG112', 'VALEH109'] )

      Ativo  Strike  bid   ask    Last  ▲ R$  ▲ % Strike  Somatório  \
0     VALE3    0.00  0.0  0.00  114.01  0.00        0.00       0.00   
1  VALEG112  114.91  0.0  1.68    1.72  0.00        0.00       0.00   
2  VALEH109  116.67  3.3  0.00    3.85  1.76        1.53       3.38   

   % sobre VALEG112  Cash  
0              0.00  0.00  
1              0.00  0.00  
2              2.94  1.62  


In [37]:
RolarOpcoes(['BBSE3', 'BBSET230', 'BBSEU230'] )

      Ativo  Strike  bid  ask   Last  ▲ R$  ▲ % Strike  Somatório  \
0     BBSE3     0.0  0.0  0.0  22.65   0.0         0.0        0.0   
1  BBSET230    23.0  0.0  1.0   0.84   0.0         0.0        0.0   
2  BBSEU230    23.0  0.8  0.0   0.96   0.0         0.0       -0.2   

   % sobre BBSET230  Cash  
0              0.00   0.0  
1              0.00   0.0  
2             -0.87  -0.2  
